In [ ]:
from ngsolve import *

from ngsolve.meshes import *

from ngsolve.solvers import *
from ngsolve.webgui import Draw

import matplotlib.pyplot as plt

from su2_yangmills import *

import numpy as np

sys.path.insert(1, '../quaternion/')

from quaternion_cf import *

In [ ]:
ne=50
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#Draw(mesh)

In [ ]:
#fesC = H1(mesh)**3 # for su2 charge
#fesA = HCurl(mesh)**3 # for su2 gauge potential
order=4
fesC = Periodic(L2(mesh, order=order))**3 # for su2 charge

fesG = Periodic(L2(mesh, order=0))**4 # for gauge transformations

In [ ]:
# set gauge transform

#phi1_CF = CF( gaussxyper([0,0], 1.) )
qphi1_CF = CF( sin(8*pi*x)*sin(8*pi*y) )
#qphi1_CF = CF( 0.5*pi )

#n1_CF = CF( (0,0,1) )
#n1_CF = CF( (0,1./sqrt(2),1./sqrt(2)) )
#n1_CF = CF( (1./sqrt(2),0,1./sqrt(2)) )
n1_CF = CF( (1./sqrt(2),1./sqrt(2),0) )


#phi_CF = CF(pi*20* x*(1-x)*y*(1-y))
phi_CF = CF(0)
#theta_CF = CF(0.1*pi*y)
theta_CF = CF(0)

n_CF = CF( (sin(theta_CF)*cos(phi_CF),sin(theta_CF)*sin(phi_CF),cos(theta_CF)) )

#phi2_CF = CF( 0 )
#phi2_CF = -1.*phi1_CF
qphi_CF = qphi1_CF

gq_CF = get_rotqCF(qphi_CF, n_CF)


gfg = GridFunction(fesG)

gfg.Set(gq_CF)

Draw(phi_CF, mesh)
# should be 1
Draw(Norm(gfg), mesh)
# Can differ
Draw(gfg[0], mesh)

In [ ]:
# set charge density
#rhoCF = CF( (rho(), 0, 0) )
rhoCF = CF( (CF(1), CF(0), 0) )

gfrho = GridFunction(fesC)

#gfrho.Set( rhoCF )
for i in range(gfrho.dim):
        gfrho.components[i].Set( rhoCF[i] )

    
#gfrho.vec.data[:] = 0.
#gfrho.vec.data[77] = 1.

    
c = rhoCF
gradc = CF( (c.Diff(x), c.Diff(y)) ).Reshape((2,3))


def Norm_CF(cf):
    return sqrt(sum( [cf[i]*cf[i] for i in range(cf.dim)] ))


#Draw(Norm(c), mesh)
#Draw(Norm(gfrho), mesh)
#Draw(gfrho.components[0], mesh)
Draw(gfrho, mesh)
#Draw(Norm(gfrho), mesh)
print(Integrate(Norm(gfrho), mesh))

In [ ]:
def gauge(f, _gfg):
    #return qmulCF(qmulCF(_gfg, f), qconjCF(_gfg))
    return vec_from_q( qmulCF(qmulCF(_gfg, q_from_vecCF( f )), qconjCF(_gfg)) )
 
gfrho_gauged = GridFunction(fesC)
gfrho_gauged.Set(gauge(gfrho, gfg))

Draw(gfrho_gauged, mesh)
#Draw(Norm(gfrho_gauged), mesh)
print(Integrate(Norm(gfrho_gauged), mesh))

In [ ]:
Draw(gfrho_gauged - gfrho, mesh)
Draw(Norm(gfrho_gauged) - Norm(gfrho), mesh)

In [ ]:
_gfrho = GridFunction(fesC)
_gfrho.Set(gauge(gfrho_gauged, qconjCF(gfg)))

Draw(gfrho - _gfrho, mesh)
Draw(Norm(gfrho) - Norm(_gfrho), mesh)